# Classes and Packages

### `! git clone https://github.com/ds4e/programming`

## Classes and Packages... by way of Matrices
- We're going to review the basics of classes, partly because it will help you understand what happens when you import packages: They're just a directory with instructions about how to load classes into your computing environment
- A matrix class objected is created by passing in
 a list of lists, and the class functions are then things like matrix multiplication or RREF
- An $ N \times K $ object is a matrix, an $ N \times 1$ object is a vector, and a $ 1 \times 1 $ object is a scalar: These are all generally called **tensors**, and the **rank** of a tensor is how many dimensions it has (scalar is 0 rank, vector is 1 rank, matrix is 2 rank). I'm going to call everything an $N \times K$ matrix.
- The goal is to introduce you to thinking about classes, error handling, and importing packages a bit more thoughtfully
- If you haven't watched the linear algebra video, go do that so you can understand what is happening
- You should use NumPy, not what we do today. But what we do today will help you understand NumPy much better.

In [1]:
A = [ [1,3,5], [2,4,6], [-1,4,8]]
print(A)

[[1, 3, 5], [2, 4, 6], [-1, 4, 8]]


## Selecting Elements of the Matrix
- We're representing the matrix as a list of lists
- How do I select the $k$-th row of the matrix, with $k=0$ corresponding to the first row? `A[k]`
- How do I select the $j$-th column of the $k$-th row of the matrix? `A[k][j]`
- If the matrix were a higher-dimensional tensor, we'd have something like `A[k][j][l][m][n]` and need rules about how to algebraically manipulate higher-dimensional objects

In [2]:
A = [ [1,3,5]]
A[0][2]

5

- You can use this basic idea for general multi-dimensional arrays and tensors:

In [3]:
A = [ [[1,3], [2,4], [-1,4]], 
     [[2,-5], [8,11], [-4,2]], 
     [[22,16], [7,11], [-32,91]] ]
print(A) # Rank 3 tensor
print(A[0]) # Rank 2 tensor: Matrix
print(A[0][1]) # Rank 1 tensor: Vector
print(A[0][1][1]) # Rank 0 tensor: Scalar

[[[1, 3], [2, 4], [-1, 4]], [[2, -5], [8, 11], [-4, 2]], [[22, 16], [7, 11], [-32, 91]]]
[[1, 3], [2, 4], [-1, 4]]
[2, 4]
4


- The problem, such as it is, is that we'd have to be much more careful in writing our algorithms to accommodate any rank of the tensor (e.g. matrix multiplication)
- This is why packages like TensorFlow and PyTorch and Keras are popular: I don't want to do what I just described, I'd much rather someone else do it in a high quality and robust way so I can focus on my application

## Downsides of this approach
- A scalar is a somewhat verbose `s = [[3]]`
- A vector is a also a bit verbose: `v = [[1,3,5]]`
- We won't be extending the matrix class past two dimensions to higher rank tensors
- It won't be computationally efficient, like NumPy
- Some things, like computing a transpose, require a bit more thought than they feel like they should
- Advanced matrix decompositions won't be available, obviously

## What do we want to implement?
- It should be easy to get the number of rows and number of columns
- We should be able to transpose the matrix
- We should be able to do scalar and matrix multiplication
- Row reduction to echelon form would be nice
- If we do RREF then why not solve systems?
- If we solve systems, why not implement some sort of inverse?
- If we can do matrix inverses, why not decompositions?
- This is called feature creep: The tendency for projects to get out of control and never reach completion

## A Class
- Let's take a quick look at the matrix class I threw together:

In [24]:
import random # To generate random matrices

class Mx:
    """ Basic version of a matrix class. """

    ############ Constructor:
    
    def __init__(self, vals, rows = 1, cols = 1):
        if vals is None: # If no values are provided...
            self.rows = rows
            self.cols = cols
            self.m = [ [0]*cols for i in range(rows)] # Actual values
        else: # If a list of lists is passed in..
            self.rows = len(vals)
            self.cols = len(vals[0])
            self.m = vals

    ############ Class methods:

    def randInt(self,lower=-1,upper=1):
        for i in range(self.rows):
            for j in range(self.cols):
                self.m[i][j] = random.randint(lower,upper)

    def randFloat(self,lower=-1,upper=1):
        for i in range(self.rows):
            for j in range(self.cols):
                self.m[i][j] = random.uniform(lower,upper)

    def eye(self):
        self.m = [ [0]*self.cols for i in range(self.rows)]
        for i in range(self.rows):
                self.m[i][i] = 1


    ############ Class Functions:

    def t(A):
        At = Mx( vals=None, rows=A.cols, cols=A.rows )
        for i in range(A.rows):
            for j in range(A.cols):
                At.m[j][i] = A.m[i][j]
        return At

    def add(A,B):
        rws_A = A.rows
        cls_B = B.cols
        C = []
        for i in range(rws_A):
                C.append( [ A.m[i][j] + B.m[i][j] for j in range(cls_B)] )
        C = Mx(C)
        return C

    def sub(A,B):
        rws_A = A.rows
        cls_B = B.cols
        C = []
        for i in range(rws_A):
                C.append( [ A.m[i][j] - B.m[i][j] for j in range(cls_B)] )
        C = Mx(C)
        return C

    def mat_mul(A,B):
        rws_A = A.rows
        cls_A = A.cols
        rws_B = B.rows
        C = []
        for i in range(rws_A):
            ith_row = []
            for j in range(cls_A):
                result = [ A.m[i][k]*B.m[j][k] for k in range(rws_B) ]
                ith_row.append( sum(result) )
            C.append(ith_row)
        C = Mx(C)
        return C
    
    def scalar_mul(s,A):
        C = []
        for i in range(A.rows):
            C.append([ s*A.m[i][j] for j in range(A.cols)])
        C = Mx(C)
        return C
    
    # def solve(A,b):
    #     return x

    ############ Operator overloading and interpreter cruft:

    def __add__(A, B): # +
        return Mx.add(A,B)
    
    def __sub__(A, B): # -
        return Mx.sub(A,B)

    def __invert__(A): # ~
        return Mx.t(A)
    
    def __mul__(A, B): # *
        A_is_mx = isinstance(A,Mx)
        B_is_mx = isinstance(B,Mx)
        if A_is_mx and B_is_mx:
            y = Mx.mat_mul( A, B )
        elif A_is_mx and not B_is_mx:
            y = Mx.scalar_mul( B, A )
        elif not A_is_mx and B_is_mx:
            y = Mx.scalar_mul( A, B )
        else:
            y = A*B
        return y

#    def __truediv__(A,b): # b/A
#        return Mx.solve(A,b)

    def __str__(self):
        # output = ''
        # for i in range(self.rows):
        #     output = output+str(self.m[i])+'\n'
        # return output
        return str(self.m)

    def __repr__(self):
        output = ''
        for i in range(self.rows):
            output = output+str(self.m[i])+'\n'
        return output


Does it "work"?

In [25]:
A = [ [1,3,5], [2,4,6], [-1,4,8]]
B = [ [-7,6,4], [-5,12,7], [8,-1,5]]

A = Mx(A)
B = Mx(B)

print(A)
print(B)

D = A*B

D

[[1, 3, 5], [2, 4, 6], [-1, 4, 8]]
[[-7, 6, 4], [-5, 12, 7], [8, -1, 5]]


[31, 66, 30]
[34, 80, 42]
[63, 109, 28]

In [26]:
A*5

[5, 15, 25]
[10, 20, 30]
[-5, 20, 40]

In [28]:
print(D)
~D

[[31, 66, 30], [34, 80, 42], [63, 109, 28]]


[31, 34, 63]
[66, 80, 109]
[30, 42, 28]

In [29]:
D.randFloat()
D

[-5.6472996100835715, 1.2516804313060383, 5.598511841314192]
[-8.05342194297056, -5.688805135107371, 4.14277060506377]
[6.423034236604259, 6.56707791987856, -7.671883152986821]

In [30]:
I = Mx(vals = None, rows=4,cols=4)
I.eye()
I

[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]

## Key Pieces
- The `__init__(self,...)` function describes how an instance is initialized or constructed. Every class has this function.
- The class methods refer to `self` and the class functions do not. 
    - When you include `self` as an argument, you can use `A.fcn()` to apply the function to object `A`. 
    - If you do not include `self`, you instead use `class.fcn(A,B)` to apply the function to class instances `A` and `B`. 
    - So, for example, `A.t()` takes the transpose of `A`, while `Mx.add(A,B)` adds `A` and `B` together.
- The Operator Overloading sections shows you how to make your class "fit in" to the Python computing environment. You can bind methods to the `+`, `-`, `*` and `/` operators, among others, to tell Python how to do computations with your new class.
    - The `*` case is interesting: I want `*` to handle scalar and matrix multiplication correctly. All of the cases are trying to make that work, and I did not quite succeed.
- The `__repr__(self)` function describes what happens when users simply type the name of a class instance into the interpreter. Here, I chose to print the matrix. Sometimes, users don't understand that the object itself is not just the repr output. It really helps to understand this.
- The `__print__(self)` function describes what happens when you type `print(A)`: This is different from `repr`, and explains why the output looks different in notebooks or the interpreter depending on what you're doing.

## Packages and Modules
- How do we make this work reusable, like the packages we pip and import?
- In this directory, there's a folder called `packageName`. Inside it, there are two files: `__init__.py` and `moduleName.py`
    - The `__init__.py` file alerts Python that this folder a package, and gives instructions about how to import its contents. Ours is empty, which is fine. More generally, you can given instructions to the Python interpreter about the package should be imported. For example, if you add `__all__ = ["something1", "something2"]` to `__init__.py`, then `from packageName import *` only imports the something's that are listed. 
    - The `moduleName.py` file includes the class, exactly as above. 
- So now we can reuse the code from our package, as follows:
    - `from packageName.moduleName import Mx`: `Mx()` is now a class in your namespace
    - `from packageName import moduleName`: Use `moduleName.Mx()`
    - `import packageName.moduleName`: Use `packageName.moduleName.Mx()`
- This also shows you how different import statements require you to interact with the module and classes inside it differently